In [1]:
import csv 
import os 
from zipcodes import pullzipcodes

####import list of NYC only zipcodes to filter out RefUSA data

In [234]:
directory = os.environ['data_path']
files = sorted(os.listdir(directory))
df10 = directory + "/" + files[4]

In [212]:
files

['Hist_Bus_2010.csv',
 'Hist_Bus_2011.csv',
 'Hist_Bus_2012.csv',
 'Hist_Bus_2013.csv',
 'Hist_Bus_2014.csv']

In [195]:
zips = pullzipcodes()

####read RefUSA data and filter by zipcodes 

In [196]:
import pandas as pd

In [235]:
ref10 = pd.read_csv(df10, dtype = {'ZIP': object, 'CENSUS_TRACT_2010': object, 'CENSUS_BG_2010': object})

In [236]:
ref10.b = ref10[ref10['ZIP'].isin(zips)]

In [93]:
# ref10.b['CENSUS_TRACT_2010']

####read Borough Boundary Files from NYC Pluto 

In [7]:
import shapefile
from pyproj import Proj, transform 
from rtree import index 

In [8]:
#sf = shapefile.Reader("nybb_15b/nybb.shp")
sf = shapefile.Reader("nycb2010_15b/nycb2010.shp")
shapes = sf.shapes()

In [9]:
sf.record(0) 
#information - cb, boro, boro_name, cen_tract, boro+cb+ct, .. , ..

['1000',
 '5',
 'Staten Island',
 '000900',
 '50009001000',
 '2.50894811457e+003',
 '2.44589603884e+005']

In [10]:
sf.fields

[('DeletionFlag', 'C', 1, 0),
 ['CB2010', 'C', 4, 0],
 ['BoroCode', 'C', 1, 0],
 ['BoroName', 'C', 32, 0],
 ['CT2010', 'C', 6, 0],
 ['BCTCB2010', 'C', 11, 0],
 ['Shape_Leng', 'F', 19, 11],
 ['Shape_Area', 'F', 19, 11]]

In [11]:
# for i in range(5):
#     print sf.record(i)
def convert(li):
    f = 0.304800
    inProj = Proj(init='epsg:2263')
    outProj = Proj(init='epsg:4326')
    x0,y0 = li[0]*f, li[1]*f
    x1,y1 = li[2]*f, li[3]*f
    
    x2,y2 = transform(inProj,outProj,x0,y0)
    x3,y3 = transform(inProj,outProj,x1,y1)
    return (x2,y2,x3,y3)

In [12]:
convert(shapes[0].bbox) # bbox provides bottom left and top right corners of any polygon 

(-74.0823760429353, 40.64297065408437, -74.07914370756991, 40.64479304833291)

####Build rtree

In [21]:
idx = index.Index()

In [22]:
loc = 0 
for i in shapes:
    corners = shapes[loc].bbox 
    left, bottom, right, top = convert(corners)
    idx.insert(loc, (left, bottom, right, top))
    loc += 1

In [25]:
def most_common(lst):
    return max(lst, key=lst.count)

####This function extracts the borough of a business

In [28]:
def findBorough(row):
    a = row["LONGITUDE_2010"]
    b = row["LATITUDE_2010"]
    
    if pd.isnull(a) == True: 
        return 'NA' 
    
    else: 
        li = list(idx.intersection((a, b)))
        if len(li) != 0:
            li2 = [sf.record(ix)[2] for ix in li]
            bor = most_common(li2)
            return bor
        else: 
            return 'NA'    

####This function extracts the census tract of a business

In [29]:
def findCT(row):
    a = row["LONGITUDE_2010"]
    b = row["LATITUDE_2010"]
    
    if pd.isnull(a) == True: 
        return 'NA' 
    else:
        li = list(idx.intersection((a, b)))
        
        if len(li) != 0:
            li2 = [sf.record(ix)[3] for ix in li]
            ct = most_common(li2)
            return ct

####This function extracts the census block of a business.  There are several values; we take the census block that occurs most in any list

In [76]:
def findCB(row):
    a = row["LONGITUDE_2010"]
    b = row["LATITUDE_2010"]
    
    if pd.isnull(a) == True: 
        return 'NA' 
    else:
        li = list(idx.intersection((a, b)))
        
        if len(li) != 0:
            li2 = [sf.record(ix)[0][0] for ix in li]
            ct = most_common(li2)
            return ct

In [77]:
pd.set_option('chained_assignment','warn')

In [237]:
ref10.b['SHP_BOROUGH'] = ref10.b.apply(findBorough, axis=1)

/opt/rh/anaconda/root/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [238]:
ref10.b['SHP_CENSUS_TRACT'] = ref10.b.apply(findCT, axis=1)

/opt/rh/anaconda/root/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [239]:
ref10.b['SHP_CENSUS_BLOCK'] = ref10.b.apply(findCB, axis=1)

/opt/rh/anaconda/root/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


####read nys_acs data (contains New York, Richmond, Kings, Queens, and Bronx Counties) 

In [87]:
acs = pd.read_csv("nys_mi.csv", dtype = {'Geo_TRACT': object, 'Geo_BLKGRP': object, 'Geo_COUNTY': object})

In [139]:
counties = ['005', '047', '061', '081', '085']

counties_dict = {'005': 'Bronx',
                 '047': 'Brooklyn',
                 '061': 'Manhattan',
                 '081': 'Queens',
                 '085': 'Staten Island'}
 
acs.b = acs[acs['Geo_COUNTY'].isin(counties)][['Geo_NAME', 'Geo_COUNTY', 'Geo_TRACT', 'Geo_BLKGRP', 'SE_T057_001']]
acs.b = acs.b.rename(columns = {'Geo_TRACT': 'SHP_CENSUS_TRACT', 'Geo_BLKGRP': 'SHP_CENSUS_BLOCK'})

In [145]:
def labelBorough(row):
    boro = counties_dict[row['Geo_COUNTY']]
    return boro

In [151]:
acs.b['SHP_BOROUGH'] = acs.b.apply(labelBorough, axis = 1)

####Merge Files 

In [240]:
result = pd.merge(ref10.b, acs.b, how = 'inner', on = ['SHP_BOROUGH', 'SHP_CENSUS_TRACT','SHP_CENSUS_BLOCK'])

####Append the LMI attribute 

##### The suggested threshold for LMI is a household income of $55,113

In [241]:
LMI_threshold = 55113 

In [242]:
def LMI(row):
    return row['SE_T057_001'] < LMI_threshold

In [243]:
result['LMI'] = result.apply(LMI, axis = 1)

In [244]:
result.to_csv("Ref14_LMI.csv")